In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Загрузка словаря из файла ===
def load_dictionary(file_path):
    dictionary = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                dictionary[key.strip().lower()] = value.strip()
    return dictionary

# Загружаем словарь из файла
food_dict = load_dictionary('global_street_food.txt')

# === Создаем vocab на основе слов из словаря ===
words = set()
for key, value in food_dict.items():
    words.update(key.split())
    words.update(value.split())

vocab = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
for i, word in enumerate(words, start=3):
    vocab[word] = i
inv_vocab = {i: w for w, i in vocab.items()}
vocab_size = len(vocab)
embedding_dim = 8
hidden_size = 16
max_len = 10  # Увеличили максимальную длину

# === Модель (остается без изменений) ===
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.rnn(x)
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, h = self.rnn(x, h)
        out = self.fc(out)
        return out, h

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        out, _ = self.decoder(tgt, h)
        return out

encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
model = Seq2Seq(encoder, decoder).to(device)

# === Генерация ответа ===
def respond_to_input(user_input):
    # Сначала проверяем точное совпадение в словаре
    user_input_lower = user_input.lower()
    if user_input_lower in food_dict:
        return f"Это блюдо из: {food_dict[user_input_lower]}"
    
    # Если точного совпадения нет, используем модель
    input_words = user_input_lower.split()
    input_idx = [vocab.get(w, vocab["<pad>"]) for w in input_words]
    input_idx = input_idx[:max_len] + [vocab["<eos>"]] + [vocab["<pad>"]] * (max_len - len(input_words) - 1)
    src = torch.tensor([input_idx[:max_len]]).to(device)
    
    h = model.encoder(src)
    inputs = torch.tensor([[vocab["<sos>"]]]).to(device)
    output_sentence = []

    for _ in range(max_len):
        out, h = model.decoder(inputs, h)
        next_token = out.argmax(-1)[:, -1]
        word = inv_vocab[next_token.item()]
        if word == "<eos>":
            break
        output_sentence.append(word)
        inputs = next_token.unsqueeze(0)

    return " ".join(output_sentence) if output_sentence else "Я не знаю, что ответить."

# === Чат-бот ===
model.eval()
with torch.no_grad():
    print("Чат-бот о уличной еде. Напишите название блюда, состав которого вы хотите узнать, или 'выход' для завершения.")
    while True:
        user_input = input("Вы: ").strip()
        if user_input.lower() in {"выход", "exit", "quit"}:
            print("Бот: До свидания! Приятного аппетита!")
            break
        
        response = respond_to_input(user_input)
        print("Бот:", response)

Чат-бот о уличной еде. Напишите название блюда, состав которого вы хотите узнать, или 'выход' для завершения.


Вы:  Hot Dog


Бот: Это блюдо из: Sausage, Bun, Mustard, Ketchup


Вы:  выход


Бот: До свидания! Приятного аппетита!
